# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
from api_keys import weather_api_key_dif

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
file = "weather_dict.csv"
df = pd.read_csv(file)
df.head()

,cities,temp,humidity,wind,clouds,lat,lon,country,date
0,Qaanaaq,20.12,75,4.72,13,77.4840,-69.3632,GL,1651085189
1,Port Alfred,60.19,88,1.01,0,-33.5906,26.8910,ZA,1651085191
2,George Town,84.13,93,3.44,20,5.4112,100.3354,MY,1651084986
3,Rikitea,77.34,72,25.21,8,-23.1203,-134.9692,PF,1651085195
4,Zafra,58.75,56,10.22,73,38.4167,-6.4167,ES,1651085196


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure

<function gmaps.maps.configure(api_key=None)>

In [4]:
# Bring in location data pairs, humidity data, define map and layer, and add layer to map

df=df.dropna()

location = df[['lat','lon']]


humidity = df['humidity'].astype(float)

fig = gmaps.figure()
heat = gmaps.heatmap_layer(location, weights=humidity)
fig.add_layer(heat)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
criteria_df = df

temp_df = criteria_df[(criteria_df['temp'] < 80) &(criteria_df['temp'] >70)]

wind_df = criteria_df[criteria_df['wind']<10]

cloud_df = criteria_df[criteria_df['clouds'] == 0]

cloud_df = cloud_df.dropna()

cloud_df

,cities,temp,humidity,wind,clouds,lat,lon,country,date
1,Port Alfred,60.19,88,1.01,0,-33.5906,26.8910,ZA,1651085191
23,Nantucket,57.34,58,18.41,0,41.2835,-70.0995,US,1651085229
25,Bredasdorp,56.68,89,2.42,0,-34.5322,20.0403,ZA,1651085232
37,Hermanus,56.80,61,5.06,0,-34.4187,19.2345,ZA,1651085256
43,Saint George,85.62,11,3.00,0,37.1041,-113.5841,US,1651085269
...,...,...,...,...,...,...,...,...,...
536,Iberia,78.96,36,6.91,0,29.9502,-91.7507,US,1651086217
554,Tomatlán,91.22,38,10.45,0,19.9333,-105.2500,MX,1651086253
555,Alpena,37.72,47,17.27,0,45.0617,-83.4327,US,1651086258
557,Cookeville,68.79,36,5.75,0,36.1628,-85.5016,US,1651086261


In [19]:
more_criteria = criteria_df.loc[(criteria_df["temp"] > 70) & (more_criteria["temp"] < 80)]
# df2=df.loc[(df['Discount'] >= 1000) & (df['Discount'] <= 2000)]
# more_criteria = more_criteria.loc[more_criteria["temp"] < 80]
more_criteria = more_criteria.loc[more_criteria["wind"] < 10]
more_criteria = more_criteria.loc[more_criteria["clouds"] == 0]

more_criteria

,cities,temp,humidity,wind,clouds,lat,lon,country,date
133,Tura,73.58,76,4.23,0,25.5198,90.2201,IN,1651085447
144,Saint-Pierre,73.08,74,8.05,0,-21.3393,55.4781,RE,1651085468
181,Avera,75.54,27,7.52,0,33.1940,-82.5271,US,1651085535
286,Mossamedes,72.23,79,4.54,0,-15.1961,12.1522,AO,1651085661
406,Madison,71.51,34,9.22,0,34.7334,-86.5667,US,1651085945
536,Iberia,78.96,36,6.91,0,29.9502,-91.7507,US,1651086217


### Hotel Map
* Create a dataframe named `hotel_df` that is the df created above only with the pertinent data.  See the prepared code below (`info_box_template`) to determine what data you will need for your final map.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels (aka "lodging") within 5000 meters.
* Utilize the Google Places API to find the first hotel search result.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [27]:
# Create dataframe and empty column
more_criteria = more_criteria[['lat','lon','cities', 'country']]
more_criteria['Hotel Name']=""
hotel_df = more_criteria
hotel_df



,lat,lon,cities,country,Hotel Name
133,25.5198,90.2201,Tura,IN,
144,-21.3393,55.4781,Saint-Pierre,RE,
181,33.1940,-82.5271,Avera,US,
286,-15.1961,12.1522,Mossamedes,AO,
406,34.7334,-86.5667,Madison,US,
536,29.9502,-91.7507,Iberia,US,


In [29]:
# Loop through dataframe and add nearby hotel result

url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {
    "radius": 5000,
    "types": "hotel",
    "key": g_key  
}


for index, row in hotel_df.iterrows():
    lat = row["lat"]
    lon = row["lon"]

    params["location"] = f"{lat},{lon}"
    
    hotel_response = requests.get(url, params=params).json()
    
    # extract results
    results = hotel_response['results']
    
    try:
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing hotel... skipping.")




hotel_df


,lat,lon,cities,country,Hotel Name
133,25.5198,90.2201,Tura,IN,Tura
144,-21.3393,55.4781,Saint-Pierre,RE,Saint Pierre
181,33.1940,-82.5271,Avera,US,Avera
286,-15.1961,12.1522,Mossamedes,AO,Namibe
406,34.7334,-86.5667,Madison,US,Huntsville
536,29.9502,-91.7507,Iberia,US,Lydia


In [31]:
# NOTE: Do not change any of the code in this cell
# Analyze the code below for variables but these should be all columns in hotel_df

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>cities</dt><dd>{cities}</dd>
<dt>country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lon"]]

In [34]:
# Add marker layer ontop of heat map
layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(layer)
fig


# Display figure



Figure(layout=FigureLayout(height='420px'))